# IA Notebook #2 - Ataques Adversarios

In [1]:
!pip install tensorflow keras

In [2]:
import tensorflow as tf
import keras

import matplotlib.pyplot as plt
import numpy as np

AttributeError: module 'tensorflow.python.keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [ ]:
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras import backend as K

In [ ]:
iv3 = InceptionV3()

In [ ]:
print(iv3.summary())

In [ ]:
from keras.preprocessing import image

x = image.load_image_to_array(image.load_image("../assets/beer.png"))

# (altura de la foto, ancho de la foto, profundidad de los píxeles - RGB en este caso (3 canales))
print(x.shape)

x = image.load_image_to_array(image.load_image("../assets/beer.png"), target_size=(299, 299))
# x = image.load_image_to_array(image.load_image("../assets/cat.png"), target_size=(299, 299))

# Cambio de rango: 0-255 -> -1-1
x /= 255
x -= 0.5
x *= 2

# (batchSize, ...otras dimensiones)
x = x.reshape([1, x.shape[0], x.shape[1], x.shape[2]])

y = iv3.predict(x)

y.shape # (1, 1000)

# decode_predictions para mapear las probabilidades a los nombres de las clases
decode_predictions(y)

In [ ]:
## Ataques adversarios

In [ ]:
# Parámetros fijos y optimización en los datos de entrada
# La red neuronal definida por Tensor Flow es un grafo de procesos que se realizan sobre los datos
# La idea es redefinir el grafo que represente el proceso de optimización para poder generar las imágenes

input_layer = iv3.layers[0].input
output_layer = iv3.layers[-1].output

target_class = 951

# Queremos maximizar el resultado de la clase {target_class}
loss = output_layer[0, target_class]

# El gradiente entre el error y la variable de entrada
gradient = K.gradients(loss, input_layer)[0]

# El modo de funcionamiento depende de la fase en la que estamos (entrenando, funcionando)
optimize_gradient = K.function([input_layer, K.learning_phase()], [gradient, loss])

cost = 0.0

# Al manipular los píxeles la red debe mantenerse en un rango de valores (intensidad de perturbación)
pert = 0.01
max_pert = x + pert
min_pert = x - pert

adversarial - np.copy(x)
while cost < 0.95:
    gr, cost = optimize_gradient([adversarial, 0]) # 0: Testing phase
    adversarial += gr
    adversarial = np.clip(adversarial, min_pert, max_pert) # np.clip?
    adversarial = np.clip(adversarial, -1, 1) # Umbrales de IV3
    print("Target cost:", cost)

In [ ]:
adversarial /= 2
adversarial += 0.5
adversarial *= 255

In [ ]:
img_array = adversarial[0].astype(np.uint8)

plt.imshow(img_array)
plt.show()

In [ ]:
from PIL import Image

im = Image.fromarray(img_array)
im.save("../assets/hacked.png")